In [ ]:
%run authenticate.ipynb
import datetime
import pandas as pd

In [ ]:
#authenticate to Strava and get all activities
client = getAuthenticatedClient()
activitiesStrava = client.get_activities()

Some simple functions to export the activities to pandas format and compute useful statistics.

In [ ]:
def getActivitiesBike(customIdToBike = True):
    
    dic = {b.id: b.name for b in client.get_athlete().bikes} #bikes are stored with an id in strava.
    
    def gearIdToBike(x, dic): #converts the gear_id to the name of the actual bike I was riding (makes sure there is no None)
        if (x['gear_id'] == None) and (x['start_date_local'] > pd.Timestamp('2024-05-24').tz_localize('UTC')):
            return 'deRosa' #my current main bike
        elif (x['gear_id'] == None) and (x['start_date_local'] <= pd.Timestamp('2024-05-24').tz_localize('UTC')):
            return 'BMC diablo' #my old bike
        else:
            return dic[x['gear_id']]
    
    features = ['name', 'id', 'start_date_local', 'distance', 'gear_id', 'sport_type', 'elapsed_time']
    
    activities = []
    for activity in activitiesStrava:
        activities.append([getattr(activity, f, None) for f in features])
    activities = pd.DataFrame(activities, columns = features)
    
    activities['distance'] = activities['distance'].apply(lambda x: x//1000) #converts to km
    
    activities = activities[(activities['sport_type'] == 'Ride') | (activities['sport_type'] == 'VirtualRide')] #filters out biking activities
    
    if customIdToBike == True: #exchanges gear_id with name of bike. 
        activities['bike'] = activities.apply(lambda x: gearIdToBike(x, dic), axis = 1)
    else:
        activities['bike'] = activities.apply(lambda x: None if x.gear_id == None else dic[x.gear_id], axis = 1)
        
    activities['type'] = activities.sport_type.apply(lambda x: 'Virtual' if x == 'VirtualRide' else 'Outdoor')
    
    activities['date'] = activities.start_date_local.apply(lambda x: x.strftime("%Y-%m-%d"))

    activities['time'] = round(activities['elapsed_time']/3600,1)
    
    return activities[['name', 'date', 'distance', 'bike', 'type', 'time']]



#relevant informations for (my) chains and cassettes in use
chainDeRosa = {
    'Ultegra': ['2024-05-24', '2025-02-18'], #each entry in the list denotes the time the chain was put on (and the other chain removed)
    'Dura Ace': ['2024-12-11']
}
CassetteDeRosa = { #two distinct cassettes, one mounted to indoor trainer, the other on the road bike.
    'Ultegra': {'type': 'Outdoor'},
    '105': {'type': 'Virtual'}
}

def getChainWear():
    activities = getActivitiesBike()
    distinct_dates = sorted(sum(list(chainDeRosa.values()),[]))
    dates = [(start, end) for start,end in zip(distinct_dates, distinct_dates[1:])] + [(distinct_dates[-1], '2222-11-11')]
    
    fullDatesChains = {}
    for c in chainDeRosa.keys():
        fullDatesChains[c] = []
        for (start,end) in dates:
            if start in chainDeRosa[c]:
                fullDatesChains[c].append((start,end))
    for c in fullDatesChains:
        chain_life = sum( activities[(start < activities.date) & (activities.date <= end) & (activities.bike == 'deRosa')].distance.sum() for (start,end) in fullDatesChains[c] )
        print(f"Chain {c}: ", chain_life, " km")

def getCassetteWear():
    activities = getActivitiesBike()
    for cass in CassetteDeRosa:
        wear = activities[(activities.type == CassetteDeRosa[cass]['type']) & (activities.bike == 'deRosa')].distance.sum()
        print(f"Cassette {cass} : {wear}, km")


In [ ]:
#returns the last 10 activities and computes the KMs this year
activities = getActivitiesBike(True) #gets the activities
print(activities[:10])
total = activities[activities.date > '2025-01-01'].distance.sum()
print('KMs this year: ', total)

In [ ]:
client.get_athlete()

In [ ]:
#shows the wear (KM) for each chain and cassette
getChainWear()
getCassetteWear()